In [ ]:
# brew install portaudio
# %pip install pyaudio soundfile
%pip install pydub

In [1]:
import pyaudio

p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

print("사용 가능한 오디오 장치 목록:")
for i in range(0, numdevices):
    device_info = p.get_device_info_by_host_api_device_index(0, i)
    print(f"Index {i}: {device_info.get('name')}, Max Input Channels: {device_info.get('maxInputChannels')}")

p.terminate()

사용 가능한 오디오 장치 목록:
Index 0: BlackHole 2ch, Max Input Channels: 2
Index 1: MacBook Pro Microphone, Max Input Channels: 1
Index 2: MacBook Pro Speakers, Max Input Channels: 0
Index 3: Microsoft Teams Audio, Max Input Channels: 1
Index 4: Multi-Output Device, Max Input Channels: 0


In [2]:
import soundfile as sf

try:
    info = sf.info('record_1.flac') # 녹음된 파일 이름으로 변경
    print(f"File Sample Rate: {info.samplerate}")
except Exception as e:
    print(f"Error reading file: {e}")

Error reading file: Error opening 'record_1.flac': System error.


In [3]:
import pyaudio

audio = pyaudio.PyAudio()

print("=" * 60)
print("사용 가능한 오디오 장치 목록:")
print("=" * 60)

blackhole_index = None

for i in range(audio.get_device_count()):
    info = audio.get_device_info_by_index(i)
    is_blackhole = "BlackHole" in info['name']
    
    marker = " <<< BlackHole 발견!" if is_blackhole else ""
    
    print(f"\n장치 {i}: {info['name']}{marker}")
    print(f"  - 최대 입력 채널: {info['maxInputChannels']}")
    print(f"  - 최대 출력 채널: {info['maxOutputChannels']}")
    print(f"  - 기본 샘플 레이트: {info['defaultSampleRate']} Hz")
    
    if is_blackhole and info['maxInputChannels'] > 0:
        blackhole_index = i
    
audio.terminate()

print("\n" + "=" * 60)
if blackhole_index is not None:
    print(f"✅ BlackHole 입력 장치: INDEX = {blackhole_index}")
    print(f"   코드에서 INPUT_DEVICE_INDEX = {blackhole_index} 로 설정하세요")
else:
    print("❌ BlackHole 입력 장치를 찾을 수 없습니다.")
print("=" * 60)

사용 가능한 오디오 장치 목록:

장치 0: BlackHole 2ch <<< BlackHole 발견!
  - 최대 입력 채널: 2
  - 최대 출력 채널: 2
  - 기본 샘플 레이트: 44100.0 Hz

장치 1: MacBook Pro Microphone
  - 최대 입력 채널: 1
  - 최대 출력 채널: 0
  - 기본 샘플 레이트: 44100.0 Hz

장치 2: MacBook Pro Speakers
  - 최대 입력 채널: 0
  - 최대 출력 채널: 2
  - 기본 샘플 레이트: 44100.0 Hz

장치 3: Microsoft Teams Audio
  - 최대 입력 채널: 1
  - 최대 출력 채널: 1
  - 기본 샘플 레이트: 48000.0 Hz

장치 4: Multi-Output Device
  - 최대 입력 채널: 0
  - 최대 출력 채널: 2
  - 기본 샘플 레이트: 44100.0 Hz

✅ BlackHole 입력 장치: INDEX = 0
   코드에서 INPUT_DEVICE_INDEX = 0 로 설정하세요


In [1]:
import pyaudio
import numpy as np
import soundfile as sf
from pydub import AudioSegment
import os
import threading
from queue import Queue

# --- 환경 설정 ---
THRESHOLD_PEAK = 650     
SILENCE_LIMIT = 2    
PAUSE_TOLERANCE_SECONDS = 3

# MP3 설정 (팟캐스트 용도)
MP3_BITRATE = "128k"  # 128kbps - 음성 품질 충분, 파일 크기 작음

CHUNK = 1024
INPUT_DEVICE_INDEX = 0  # 🔴 위에서 확인한 BlackHole INDEX로 변경하세요!

# --- PyAudio 초기화 및 장치 정보 확인 ---
audio = pyaudio.PyAudio()

# 실제 장치의 정보 가져오기
device_info = audio.get_device_info_by_index(INPUT_DEVICE_INDEX)
print(f"\n🎙️  사용 중인 장치: {device_info['name']}")
print(f"📊 기본 샘플 레이트: {device_info['defaultSampleRate']} Hz")
print(f"🔊 최대 입력 채널: {device_info['maxInputChannels']}")

# 장치의 실제 설정 사용 (BlackHole은 보통 48000Hz)
RATE = int(device_info['defaultSampleRate'])
CHANNELS = min(2, device_info['maxInputChannels'])
FORMAT = pyaudio.paInt16

print(f"\n✅ 녹음 설정: {RATE}Hz, {CHANNELS}채널, 16bit")
print(f"🎵 MP3 변환 설정: {MP3_BITRATE}")

CHUNK_DURATION = CHUNK / RATE 
PAUSE_TOLERANCE_CHUNKS = int(PAUSE_TOLERANCE_SECONDS / CHUNK_DURATION)

# --- MP3 변환 작업 큐 및 스레드 ---
conversion_queue = Queue()
keep_converting = True

def mp3_converter_thread():
    """백그라운드에서 MP3 변환을 처리하는 스레드"""
    while keep_converting:
        try:
            flac_file, mp3_file = conversion_queue.get(timeout=1)
            
            print(f"\n🔄 MP3 변환 중: {flac_file} → {mp3_file}")
            
            try:
                audio_data = AudioSegment.from_file(flac_file, format="flac")
                audio_data.export(mp3_file, format="mp3", bitrate=MP3_BITRATE)
                os.remove(flac_file)  # 임시 FLAC 파일 삭제
                print(f"✅ MP3 변환 완료: {mp3_file}")
            except Exception as e:
                print(f"❌ MP3 변환 실패: {e}")
                
            conversion_queue.task_done()
        except:
            continue

# 변환 스레드 시작 (데몬 스레드로 설정하여 프로그램 종료 시 자동 종료)
converter = threading.Thread(target=mp3_converter_thread, daemon=True)
converter.start()

# --- PyAudio 스트림 설정 ---
try:
    stream = audio.open(
        format=FORMAT, 
        channels=CHANNELS,
        rate=RATE, 
        input=True,
        input_device_index=INPUT_DEVICE_INDEX,
        frames_per_buffer=CHUNK
    )
except Exception as e:
    print(f"\n❌ 오류: {e}")
    print("BlackHole 장치 INDEX가 올바른지 확인하세요!")
    audio.terminate()
    exit(1)

# --- 상태 변수 ---
recording = False
frames = []
silence_counter = 0     
counter = 344  # 녹음 파일 번호 시작점
pause_chunk_count = 0

print(f"\n🎵 녹음을 시작하려면 소리를 재생하세요 (Peak > {THRESHOLD_PEAK}).")
print(f"⏸️  멈추려면 소리를 {SILENCE_LIMIT}초 이상 끊으세요.")
print(f"⌨️  종료하려면 Ctrl+C를 누르세요.\n")

try:
    while True:
        # 항상 정확히 CHUNK만큼 읽기
        raw_data = stream.read(CHUNK, exception_on_overflow=False)
        data = np.frombuffer(raw_data, dtype=np.int16)
        
        volume_peak = np.max(np.abs(data))
        queue_size = conversion_queue.qsize()
        print(f"Peak: {volume_peak:5d} | 변환 대기중: {queue_size}", end='\r') 
        
        if volume_peak > THRESHOLD_PEAK:
            silence_counter = 0
            pause_chunk_count = 0 
            
            if not recording:
                print(f"\n🔴 녹음 시작: record_{counter}.mp3 (Peak: {volume_peak})")
                recording = True
                frames = []          
                
            frames.append(data)
            
        else: 
            if recording:
                if pause_chunk_count < PAUSE_TOLERANCE_CHUNKS:
                    pause_chunk_count += 1
                    frames.append(data)
                else:
                    silence_counter += CHUNK_DURATION
                    frames.append(data)
        
        # 무음 시간 초과 체크 및 녹음 종료
        if recording and silence_counter > SILENCE_LIMIT:
            frames_to_remove = int((PAUSE_TOLERANCE_SECONDS + SILENCE_LIMIT) / CHUNK_DURATION)
            flac_filename = f"record_{counter}_temp.flac"
            mp3_filename = f"record_{counter}.mp3"
            
            if len(frames) > frames_to_remove:
                data_to_save = np.concatenate(frames[:-frames_to_remove])
            else:
                data_to_save = np.concatenate(frames) 
            
            # 스테레오인 경우 reshape 필요
            if CHANNELS == 2:
                data_to_save = data_to_save.reshape(-1, 2)
            
            min_save_time = 0.5
            duration = len(data_to_save) / RATE
            
            if duration > min_save_time:
                # FLAC으로 임시 저장
                sf.write(flac_filename, data_to_save, RATE, subtype='PCM_16')
                print(f"\n💾 녹음 완료: {flac_filename} (길이: {duration:.2f}초)")
                print(f"   샘플레이트: {RATE}Hz | 채널: {CHANNELS}ch | 비트레이트: {MP3_BITRATE}")
                
                # MP3 변환 작업을 큐에 추가 (백그라운드 처리)
                conversion_queue.put((flac_filename, mp3_filename))
            else:
                print(f"\n⚠️  녹음된 소리가 너무 짧습니다 ({duration:.2f}초 < {min_save_time}초). 저장 건너뛰기.")
                
            # 상태 초기화
            counter += 1
            recording = False
            frames = [] 
            silence_counter = 0
            pause_chunk_count = 0

except KeyboardInterrupt:
    print("\n\n⏹️  녹음 프로세스 종료.")
    print("⏳ 진행 중인 MP3 변환을 기다리는 중...")
    
    # 남은 변환 작업 완료 대기
    keep_converting = False
    conversion_queue.join()
    
finally:
    if stream.is_active():
        stream.stop_stream()
        stream.close()
    audio.terminate()
    print("👋 프로그램을 종료합니다.")


🎙️  사용 중인 장치: BlackHole 2ch
📊 기본 샘플 레이트: 44100.0 Hz
🔊 최대 입력 채널: 2

✅ 녹음 설정: 44100Hz, 2채널, 16bit
🎵 MP3 변환 설정: 128k

🎵 녹음을 시작하려면 소리를 재생하세요 (Peak > 650).
⏸️  멈추려면 소리를 2초 이상 끊으세요.
⌨️  종료하려면 Ctrl+C를 누르세요.

Peak: 10481 | 변환 대기중: 0
🔴 녹음 시작: record_344.mp3 (Peak: 10481)
Peak:   395 | 변환 대기중: 0
💾 녹음 완료: record_344_temp.flac (길이: 251.40초)
   샘플레이트: 44100Hz | 채널: 2ch | 비트레이트: 128k
Peak:   371 | 변환 대기중: 1
🔄 MP3 변환 중: record_344_temp.flac → record_344.mp3
Peak:   690 | 변환 대기중: 0
🔴 녹음 시작: record_345.mp3 (Peak: 690)
✅ MP3 변환 완료: record_344.mp3
Peak:     1 | 변환 대기중: 0
💾 녹음 완료: record_345_temp.flac (길이: 572.98초)
   샘플레이트: 44100Hz | 채널: 2ch | 비트레이트: 128k
Peak:     1 | 변환 대기중: 1
🔄 MP3 변환 중: record_345_temp.flac → record_345.mp3
Peak:  9410 | 변환 대기중: 0
🔴 녹음 시작: record_346.mp3 (Peak: 9410)
✅ MP3 변환 완료: record_345.mp3
Peak:     1 | 변환 대기중: 0
💾 녹음 완료: record_346_temp.flac (길이: 481.91초)
   샘플레이트: 44100Hz | 채널: 2ch | 비트레이트: 128k
Peak:     1 | 변환 대기중: 1
🔄 MP3 변환 중: record_346_temp.flac → record_346.mp3
Peak:  